In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

In [2]:
spark_home = os.environ.get('SPARK_HOME', None)

In [3]:
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [4]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t")).cache()

In [5]:
rdd.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [6]:
hist_all = rdd.map(lambda x: (x[2],1))\
    .reduceByKey(lambda x,y:(x+y))\
    .sortByKey()\
    .map(lambda x: x[1])

In [7]:
hist_all.take(5)

[6110, 11370, 27145, 34174, 21201]

In [8]:
filmid = '96'

In [9]:
hist_film = rdd.filter(lambda x: x[1] == filmid)\
    .map(lambda x: (x[2], 1))\
    .reduceByKey(lambda x,y:(x+y))\
    .sortByKey()\
    .map(lambda x: x[1])

In [10]:
hist_film.take(5)

[6, 20, 43, 123, 103]

In [11]:
import json

In [13]:
result = dict(hist_film = hist_film.collect(), hist_all = hist_all.collect())

In [14]:
result

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [15]:
with open("/data/home/andrey.blednykh/lab01.json", "w") as f:
    f.write(json.dumps(result))

In [16]:
sc.stop()